In [30]:
import datasource
data = datasource.get_data()
train_index, test_index = datasource.get_index_splited_by_time(data)
train_dataset = datasource.FSRDataset(data['FSR_for_force'], data['force'], train_index)

In [31]:
data

Category  id time force                             x_coord      ...  \
Position              A    B     C      D    E    F       A   B  ...   
0          0    0   0.0  0.0   0.0   22.0  0.0  0.0     NaN NaN  ...   
1          0    1   0.0  0.0   0.0  277.0  0.0  0.0     NaN NaN  ...   
2          0    2   0.0  0.0   0.0  488.0  0.0  0.0     NaN NaN  ...   
3          0    3   0.0  0.0   0.0  501.0  0.0  0.0     NaN NaN  ...   
4          0    4   0.0  0.0   0.0  540.0  0.0  0.0     NaN NaN  ...   
...       ..  ...   ...  ...   ...    ...  ...  ...     ...  ..  ...   
52144     72  635   0.0  0.0   0.0  983.0  0.0  0.0     NaN NaN  ...   
52145     72  636   0.0  0.0   0.0  962.0  0.0  0.0     NaN NaN  ...   
52146     72  637   0.0  0.0   0.0  910.0  0.0  0.0     NaN NaN  ...   
52147     72  638   0.0  0.0   0.0  851.0  0.0  0.0     NaN NaN  ...   
52148     72  639   0.0  0.0  10.0  839.0  0.0  0.0     NaN NaN  ...   

Category FSR_for_force                  FSR_for_coord                 \
Position             C      D    E    F             A    B         C   
0                  0.0    0.0  0.0  0.0           0.0  0.0  0.000000   
1                  0.0   60.0  0.0  0.0           0.0  0.0  0.000000   
2                  0.0   73.0  0.0  0.0           0.0  0.0  0.000000   
3                  0.0   84.0  0.0  0.0           0.0  0.0  0.000000   
4                  0.0  100.0  0.0  0.0           0.0  0.0  0.000000   
...                ...    ...  ...  ...           ...  ...       ...   
52144             37.0  180.0  0.0  0.0           0.0  0.0  0.587302   
52145             38.0  172.0  0.0  0.0           0.0  0.0  0.603175   
52146             38.0  165.0  0.0  0.0           0.0  0.0  0.603175   
52147             39.0  160.0  0.0  0.0           0.0  0.0  0.619048   
52148             39.0  156.0  0.0  0.0           0.0  0.0  0.619048   

Category                      
Position         D    E    F  
0         0.000000  0.0  0.0  
1         0.952381  0.0  0.0  
2         1.158730  0.0  0.0  
3         1.333333  0.0  0.0  
4         1.587302  0.0  0.0  
...            ...  ...  ...  
52144     2.857143  0.0  0.0  
52145     2.730159  0.0  0.0  
52146     2.619048  0.0  0.0  
52147     2.539683  0.0  0.0  
52148     2.476190  0.0  0.0  

[52149 rows x 32 columns]

In [33]:
import sklearn.preprocessing
train_index, test_index = datasource.get_index_splited_by_time(data)
scaler_X = sklearn.preprocessing.MinMaxScaler()
scaler_y = sklearn.preprocessing.MinMaxScaler()
index_X = 'FSR_for_force'
index_y = 'force'
data_X = data[index_X]
data_y = data[index_y]
# data.loc[:, index_X] = scaler_X.fit_transform(data_X[train_index])
# data.loc[:, index_y] = scaler_y.fit_transform(data_y[train_index])
# data_X[train_index]
import numpy as np
import pandas as pd
data.loc[np.concatenate(train_index)]
# train_index[0].union([train_index[1], train_index[2]])

Category  id time force                             x_coord      ...  \
Position              A    B    C       D    E    F       A   B  ...   
0          0    0   0.0  0.0  0.0    22.0  0.0  0.0     NaN NaN  ...   
1          0    1   0.0  0.0  0.0   277.0  0.0  0.0     NaN NaN  ...   
2          0    2   0.0  0.0  0.0   488.0  0.0  0.0     NaN NaN  ...   
3          0    3   0.0  0.0  0.0   501.0  0.0  0.0     NaN NaN  ...   
4          0    4   0.0  0.0  0.0   540.0  0.0  0.0     NaN NaN  ...   
...       ..  ...   ...  ...  ...     ...  ...  ...     ...  ..  ...   
51984     72  475   0.0  0.0  0.0  1200.0  0.0  0.0     NaN NaN  ...   
51985     72  476   0.0  0.0  0.0  1229.0  0.0  0.0     NaN NaN  ...   
51986     72  477   0.0  0.0  0.0  1248.0  0.0  0.0     NaN NaN  ...   
51987     72  478   0.0  0.0  0.0  1247.0  0.0  0.0     NaN NaN  ...   
51988     72  479   0.0  0.0  0.0  1253.0  0.0  0.0     NaN NaN  ...   

Category FSR_for_force                  FSR_for_coord                 \
Position             C      D    E    F             A    B         C   
0                  0.0    0.0  0.0  0.0           0.0  0.0  0.000000   
1                  0.0   60.0  0.0  0.0           0.0  0.0  0.000000   
2                  0.0   73.0  0.0  0.0           0.0  0.0  0.000000   
3                  0.0   84.0  0.0  0.0           0.0  0.0  0.000000   
4                  0.0  100.0  0.0  0.0           0.0  0.0  0.000000   
...                ...    ...  ...  ...           ...  ...       ...   
51984              0.0  223.0  0.0  0.0           0.0  0.0  0.000000   
51985             37.0  230.0  0.0  0.0           0.0  0.0  0.587302   
51986             39.0  236.0  0.0  0.0           0.0  0.0  0.619048   
51987             38.0  240.0  0.0  0.0           0.0  0.0  0.603175   
51988             36.0  241.0  0.0  0.0           0.0  0.0  0.571429   

Category                      
Position         D    E    F  
0         0.000000  0.0  0.0  
1         0.952381  0.0  0.0  
2         1.158730  0.0  0.0  
3         1.333333  0.0  0.0  
4         1.587302  0.0  0.0  
...            ...  ...  ...  
51984     3.539683  0.0  0.0  
51985     3.650794  0.0  0.0  
51986     3.746032  0.0  0.0  
51987     3.809524  0.0  0.0  
51988     3.825397  0.0  0.0  

[39079 rows x 32 columns]

In [8]:
data[[('force', 'A'), ('force', 'B')]]

Category force     
Position     A    B
0          0.0  0.0
1          0.0  0.0
2          0.0  0.0
3          0.0  0.0
4          0.0  0.0
...        ...  ...
52144      0.0  0.0
52145      0.0  0.0
52146      0.0  0.0
52147      0.0  0.0
52148      0.0  0.0

[52149 rows x 2 columns]

In [17]:
class UserException1(Exception):
    pass
class UserException2(Exception):
    pass
class UserException3(Exception):
    pass

try:
    a = int(input())
    if a == 1:
        raise UserException1
    elif a == 2:
        raise UserException2
    elif a == 3:
        raise UserException3
except Exception as e:
    if isinstance(e, UserException1) or isinstance(e, UserException2):
        print('exception1 or exception2')
    elif isinstance(e, UserException3):
        print('exception3')
    print('common exception')


In [1]:
from typing import Optional, Dict, Any
from ray.tune.search.optuna import OptunaSearch
from ray.tune.search import ConcurrencyLimiter
from ray import tune
import time
from ray.air import session
import ray
import ray.tune.search.optuna.optuna_search

In [2]:
def define_by_run_func(trial:ray.tune.search.optuna.optuna_search._OptunaTrialSuggestCaptor) -> Optional[Dict[str, Any]]:
    """Define-by-run function to create the search space.

    Ensure no actual computation takes place here. That should go into
    the trainable passed to ``Tuner()`` (in this example, that's
    ``objective``).

    For more information, see https://optuna.readthedocs.io/en/stable\
    /tutorial/10_key_features/002_configurations.html

    This function should either return None or a dict with constant values.
    """

    activation = trial.suggest_categorical("activation", ["relu", "tanh"])

    # Define-by-run allows for conditional search spaces.
    if activation == "relu":
        trial.suggest_float("size", -100, 100)
        trial.suggest_float("si/ze/width", 0, 20)
        trial.suggest_float("size/height", -100, 100)
    else:
        trial.suggest_float("si/ze/width", -1, 21)
        trial.suggest_float("size", -101, 101)
        
    # Return all constants in a dictionary.
    return {"steps": 100}

In [3]:
searcher = OptunaSearch(define_by_run_func, metric="mean_loss", mode="min")
algo = ConcurrencyLimiter(searcher, max_concurrent=4)

[I 2023-07-01 20:05:22,228] A new study created in memory with name: optuna


In [4]:
def evaluate(step, width, height, activation):
    time.sleep(0.1)
    activation_boost = 10 if activation=="relu" else 0
    return (0.1 + width * step / 100) ** (-1) + height * 0.1 + activation_boost

In [5]:
def objective(config):
    for step in range(config["steps"]):
        score = evaluate(step, config["si"]["ze"]["width"], config["size"]["height"], config["activation"])
        session.report({"iterations": step, "mean_loss": score})

In [6]:
num_samples = 10
tuner = tune.Tuner(
    objective,
    tune_config=tune.TuneConfig(
        search_alg=searcher,
        num_samples=num_samples,
    ),
)
results = tuner.fit()

2023-07-01 20:05:24,588	INFO worker.py:1627 -- Started a local Ray instance. View the dashboard at 127.0.0.1:8265 
2023-07-01 20:05:26,439	INFO tune.py:226 -- Initializing Ray automatically. For cluster usage or custom Ray initialization, call `ray.init(...)` before `Tuner(...)`.


TypeError: Cannot unflatten dict due the key 'size/height' having a parent key 'size', which value is not of type <class 'dict'> (got <class 'float'>). Change the key names to resolve the conflict.

In [ ]:
print("Best hyperparameters for loss found were: ", results.get_best_result("mean_loss", "min").config)

Best hyperparameters for loss found were:  {'activation': 'tanh', 'si': {'ze': {'width': 17.423511459536684}}, 'size': {'height': -79.26626664441896}, 'steps': 100}


In [5]:
import torch
class ANN(torch.nn.Module):
    def __init__(self, input_size, hidden_size, num_layer, output_size):
        super().__init__()
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.num_layer = num_layer
        self.output_size = output_size
        layers = [[hidden_size, hidden_size] for i in range(num_layer)]
        layers[0][0] = input_size
        layers[-1][1] = output_size
        self.layers = torch.nn.Sequential(
            torch.nn.Linear(layer[0], layer[1]) for layer in layers
        )
    
    def forward(self, x):
        x = self.layers(x)
        return x

In [6]:
ANN(3, 5, 3, 3)

TypeError: ANN.__init__.<locals>.<genexpr> is not a Module subclass